# 量脉调度器

*版权所有 (c) 2021 百度量子计算研究所，保留所有权利。*

## 内容概要

本教程将介绍如何使用**量脉调度器**为量子电路生成脉冲序列。本教程的内容概要如下：
- 背景介绍
- 脉冲调度规则介绍
- 示例：通过使用**量脉调度器**生成实现 $W$-量子态的脉冲
- 总结

## 背景介绍

**量脉调度器**能够为给定的量子电路自动生成高保真度的脉冲序列，以执行特定的量子计算任务。

**量脉调度器**具有以下优点：

- 高度自动化：自动产生高保真脉冲，同时最大限度地减少门操作的时间。

- 实用性：它考虑了超导量子系统的具体限制，包括能级泄漏以及退相干导致的误差等。

- 灵活性：用户可以自由定制量子比特和电路参数。同时，也可以很容易地扩展到其他量子计算平台。

**量脉调度器**实现了以下目标：

- 生成参数和任意波形发生器 AWG 输入信号阵列，以便在考虑泄漏到状态 $|2\rangle$ 时获得保真度优化的脉冲。

- 能够调度脉冲最小化闲置时间，从而减少退相干损耗。

- 支持脉冲序列的可视化。

CNOT 门通常不能在超导量子芯片上直接实现，而是通过将单量子比特门和双量子比特门（如 CR 门、CZ 门或 ISWAP 门）组成构建，而这些门可以很容易地在超导芯片上实现（通常称为原生门）。类 transmon 超导量子比特结构中常用的两比特门可分为两类：

**通量控制**

这类门具有门时间短的优点，可以最大限度地减小退相干误差。然而调整量子比特频率会引入磁通噪声，同时还会导致频率拥挤的问题。

**全微波控制**

CR 门可以使用全微波控制实现，这减轻了磁通噪声所带来的影响。然而，其缺点是单个门操作的时间较长，受退相干效应的影响较大，从而限制了实际门保真度。

![VQE](figures/cnot-gate.png) 

由于 CZ 门仅需使用两个单量子比特门就可以组合出 CNOT 门，因此**量脉调度器**使用 CZ 门来实现 CNOT 门的构造。

## 示例：通过使用**量脉调度器**生成实现 $W$-量子态的脉冲

2000 年，Dür 等人发现三体纠缠态可以以不同于 GHZ（Greenberger-Horne-Zeilinger）态的方式纠缠，他们称之为 $W$-量子态：

$$
|\Phi_{\rm GHZ}\rangle = \frac{1}{\sqrt{2}} (|000\rangle + |111\rangle),
$$

$$
|\Phi_{\rm W}\rangle = \frac{1}{\sqrt{3}} (|001\rangle + |010\rangle + |100\rangle).
$$

与 GHZ 态不同，$W$-量子态是三个本征态的叠加态，每个本征态中只有一个粒子处于激发态。$W$-量子态在其中一个粒子退相干的情况下，也可以保持最大的两体纠缠态。因而 $W$-态对于噪声具有很高的鲁棒性，在量子通信领域内备受关注。

在本教程中，我们演示了使用**量脉调度器**生成 $W$-态的方法。在开始主要步骤之前，我们从量脉（Quanlse）中引入以下包:：

In [ ]:
from Quanlse.QOperation.FixedGate import X, CNOT
from Quanlse.QOperation.RotationGate import RY
from Quanlse.Simulator.PulseSim3Q import pulseSim3Q
from Quanlse.Scheduler.Superconduct.PipelineCenterAligned import centerAligned

在量脉中实例化一个 `simulator` 对象前，我们通常需要先定义系统的配置。但在该教程中，我们可以直接使用量脉提供的预设模拟器 `pulseSim3Q`, 更多细节请参阅[多比特含噪模拟器](https://quanlse.baidu.com/#/doc/tutorial-multi-qubit-noisy-simulator)。在该模拟器中，系统的参数是固定的（如比特频率，耦合强度等）。这里我们将使用一个结构如下的系统（比特的失谐均为 $0.33*(2*\pi)\ \textrm{GHz}$）：

![qubitMap](figures/qubitMap.png)


首先，我们将定义 AWG 的采样时间 `dt`。然后在实例化 `pulseSim3Q` 时，将其传入参数当中。

In [ ]:
# Create pulseSim3Q object
dt = 0.01  # AWG sampling time
model = pulseSim3Q(frameMode='lab', dt=dt)
model.savePulse = False
model.pipeline.addPipelineJob(centerAligned)

本示例的关键步骤是构造完成指定量子任务所需的逻辑量子电路。在本例中，我们使用以下电路生成 $W$-量子态：

![VQE](figures/w-circuit.png) 

其中，

$$
\begin{align*}
R_1 &= \frac{1}{\sqrt{3}} \begin{bmatrix} \sqrt{2} & -1 \\ 1 & \sqrt{2} \end{bmatrix}, \\
R_2 &= \begin{bmatrix} \cos(\pi/8) & -\sin(\pi/8) \\ \sin(\pi/8) & \cos(\pi/8) \end{bmatrix}, \\
R_3 &= \begin{bmatrix} \cos(\pi/8) & \sin(\pi/8) \\ -\sin(\pi/8) & \cos(\pi/8) \end{bmatrix}.
\end{align*}
$$

如前所述，在超导量子系统中，CZ 门是一个原生门，我们只需再使用两个 Hadamard 门就可以构造一个 CNOT 门。其中，两个 Hadamard 门应该作用在目标比特上，即：

![VQE](figures/cnot-gate-hadamard.png) 

在本示例中，我们将使用此分解方法来构造 CNOT 门。在**量脉调度器**中，逻辑量子电路可以由以下代码定义：

In [ ]:
# Define circuit

# R1
RY(-1.231)(model.Q[0])

# X gate
X(model.Q[0])

# CNOT: 0 -> 1
CNOT(model.Q[0], model.Q[1])

# X gate
X(model.Q[0])

# R2
RY(-0.785)(model.Q[2])

# CNOT: 1 -> 2
CNOT(model.Q[1], model.Q[2])

# R3
RY(0.785)(model.Q[2])

# CNOT: 2 -> 1
CNOT(model.Q[2], model.Q[1])

注意，我们使用 `U(theta, phi, lambda)` 来分解旋转门 $R_1, R_2$ 和 $R_3$，其中：

$$
U(\theta, \phi, \lambda) = e^{i(\phi/2+\lambda/2)} R_z(\phi) R_y(\theta) R_z(\lambda) =
\begin{bmatrix} 
    \cos(\theta/2) & - e^{i\lambda} \sin(\theta/2) \\
    e^{i\phi} \sin(\theta/2) & e^{i(\phi + \lambda)} \cos(\theta/2)
\end{bmatrix}.
$$

我们可以通过 `Schedule()` 方法生成所需量子门的脉冲序列，并根据逻辑量子电路自动调度脉冲序列。`Schedule()` 方法返回的为一个 `Qjob` 对象，该对象包含了脉冲序列的所有信息。我们可以使用 `plot()` 函数生成可视化的脉冲图。最后，我们可以通过使用函数 `runHamiltonian()` 获得模拟得到的酉演化矩阵。该函数的参数为一个 `QHam` 对象以及一个 `Qjob` 对象，分别为系统和脉冲的配置。

In [ ]:
# Schedule
job = model.schedule()
job.plot(color=['red', 'green'])
finalState = model.simulate(job=job)[0]["state"].T[0]

通过运行 `runHamiltonian()` 方法，我们可以得到上面的脉冲序列。当用户在本地端运行该程序时，量脉云服务将返回量子电路中每一个量子门的保真度。最后，我们使用演化得到的时序演化算符计算每个本征态的概率。我们可以看到下面输出的量子态非常接近于一个理想的 $W$ 态，其中其他态上的概率是因为能级泄露，串扰等噪声所导致的。

In [ ]:
# Calculate final state
popList = [abs(item ** 2) for item in finalState]

# Import the operator for generating basis string list
from Quanlse.Utils.Functions import computationalBasisList

# Import the function for plot bar figures
from Quanlse.Utils.Plot import plotBarGraph

# Plot the population of computational basis
basis = computationalBasisList(3, 3)
plotBarGraph(basis, popList, "Population of a W state generated by QuanlseScheduler",
             "Computational Basis", "Population")

## 总结
您可以点击以下链接 [tutorial-scheduler.ipynb](https://github.com/baidu/Quanlse/blob/main/Tutorial/CN/tutorial-scheduler-cn.ipynb) 下载该教程的 Jupyter Notebook 文件，以在自己的计算机上运行上述程序。我们鼓励用户使用量脉调度器去实现不同的量子电路。

## 参考文献

\[1\] [Krantz, Philip, et al. "A quantum engineer's guide to superconducting qubits." *Applied Physics Reviews* 6.2 (2019): 021318.](https://aip.scitation.org/doi/abs/10.1063/1.5089550)

\[2\] https://en.wikipedia.org/wiki/Quantum_optimization_algorithms

\[3\] https://en.wikipedia.org/wiki/Quantum_algorithm

\[4\] [Dür, Wolfgang, Guifre Vidal, and J. Ignacio Cirac. "Three qubits can be entangled in two inequivalent ways." *Physical Review A* 62.6 (2000): 062314.](https://doi.org/10.1103/PhysRevA.62.062314)

\[5\] [Guo, Guang-Can, and Yong-Sheng Zhang. "Scheme for preparation of the W state via cavity quantum electrodynamics." *Physical Review A* 65.5 (2002): 054302.](https://doi.org/10.1103/PhysRevA.65.054302)